<font color='green'>***Installation and Libraries Import***</font>
---
--- 

In [1]:
# %pip install flwr
# %pip install ray 
# %pip install --upgrade pip
# %pip install torch torchvision matplotlib
# %pip install timeout
# %pip install numpy
# %pip install pandas
# %pip install datasets
# %pip install scikit-learn
# %pip install requests
# %pip install transformers accelerate bitsandbytes
# %pip install peft datasets


In [1]:
import os
import flwr as fl
import torch, ray, pandas, sklearn
import pickle
from pprint import pprint


# print("All modules loaded successfully!")
# print("FLWR version:", fl.__version__)
# print("Ray version:", ray.__version__)
# print("Torch version:", torch.__version__)

# libraries_to_uninstall = [
#     "tb-nightly==2.18.0a20240701",
#     "tensorboard==2.16.2",
#     "tensorboard-data-server==0.7.2",
#     "tensorboard-plugin-wit==1.8.1",
#     "tensorflow==2.16.2",
#     "tensorflow-io-gcs-filesystem==0.37.0",
#     "termcolor==2.4.0",
#     "terminado==0.18.1",
#     "tf-estimator-nightly==2.8.0.dev2021122109",
#     "tf_keras-nightly==2.18.0.dev2024070109",
#     "tf-nightly==2.18.0.dev20240626"
# ]
# for library in libraries_to_uninstall:
#     os.system(f"pip uninstall -y {library}")
# print("All modules loaded successfully!")
# print("FLWR version:", fl.__version__)
# print("Ray version:", ray.__version__)
# print("Torch version:", torch.__version__)

In [2]:
from collections import OrderedDict
from typing import Dict, List, Optional, Tuple
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from datasets.utils.logging import disable_progress_bar
from torch.utils.data import DataLoader, random_split, Subset
import torch.optim as optim
from torch.utils.data import Dataset, WeightedRandomSampler
import flwr as fl
from flwr.common import Metrics
# from flwr_datasets import FederatedDataset
from sklearn.preprocessing import MinMaxScaler    
from sklearn.model_selection import train_test_split
import random
import math
import pickle
import csv
import copy
print(fl.__version__)
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)

1.5.0
cuda:0


<font color='Brown'>***FL Constants***</font>
---
--- 

In [ ]:
# DEVICE
NUM_CLIENTS = 10 #48
ROUNDS = 40 #40 
BATCH_SIZE = 64
LEARNING_RATE = 0.0018
EPOCHS = 4
DATA_GROUPS = 120
BATCH_ROUND = 6
SIZE_ROUND = int(BATCH_ROUND * BATCH_SIZE * NUM_CLIENTS)
NUM_ATKS = 4
PATH = f'LLM{NUM_ATKS}atk_{ROUNDS}_rounds_{NUM_CLIENTS}_clients_{EPOCHS}_epochs_{BATCH_SIZE}_batch_{LEARNING_RATE}_lr_{DATA_GROUPS}_data_groups/'
G=0

In [5]:
# ============================================================
# ✅ Quiet Mode Setup — Silence Background Logs Completely
# ============================================================
import os
import logging
import warnings

# -- 1️⃣ Disable parallel tokenizer & Ray telemetry noise --
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["RAY_USAGE_STATS_ENABLED"] = "0"
os.environ["RAY_USAGE_STATS_OUTPUT_ENABLED"] = "0"
os.environ["RAY_DISABLE_DASHBOARD"] = "1"
os.environ["RAY_BACKEND_LOG_LEVEL"] = "error"
os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "1"
os.environ["TRANSFORMERS_VERBOSITY"] = "error"  # silence Hugging Face pipeline info

# -- 2️⃣ Suppress all non-critical Python warnings --
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

# -- 3️⃣ Silence common noisy libraries --
for lib in [
    "flwr",          # Flower federated server/client
    "ray",           # Ray backend
    "transformers",  # HF transformers logs
    "torch",         # PyTorch initialization logs
    "datasets",      # HF datasets
    "urllib3",       # Networking / SSL warnings
    "filelock",      # FileLock for model caching
    "absl",          # TensorFlow-style logs if imported
]:
    logging.getLogger(lib).setLevel(logging.CRITICAL)

# -- 4️⃣ Optional: quiet matplotlib/font/system noise (if present)
logging.getLogger("matplotlib").setLevel(logging.CRITICAL)
logging.getLogger("PIL").setLevel(logging.CRITICAL)

# -- 5️⃣ Confirm quiet mode
print("✅ Quiet mode enabled — only print() outputs and LLM feedback will appear.")


✅ Quiet mode enabled — only print() outputs and LLM feedback will appear.


<font color='Light Blue'>***Dataset Preparations***</font>
---
--- 

In [6]:
TrafficData = {}
TrafficData['Dataset']={}
sets_names = ['30','100','70','50','testing']
for  DATA_NUM in sets_names:
    TrafficData['Dataset'][DATA_NUM]=pd.read_csv(f'data/2_Dataset_{NUM_ATKS}_Attack_{DATA_NUM}_normal.csv', low_memory=False, quoting=csv.QUOTE_NONE, on_bad_lines='skip')
    print(DATA_NUM, TrafficData['Dataset'][DATA_NUM].shape)
for DATA_NUM in TrafficData['Dataset']:
    TrafficData['Dataset'][DATA_NUM]=TrafficData['Dataset'][DATA_NUM].sample(frac=1, random_state=42).reset_index(drop=True)

30 (184320, 99)
100 (184320, 99)


KeyboardInterrupt: 

In [ ]:
# === Combine TrafficData splits using transpose for interleaving ===
TrafficData['Split'] = {}
sets_training = ['30', '100', '70', '50']

# Split each dataset into DATA_GROUPS
for DATA_NUM in sets_training:
    TrafficData['Split'][DATA_NUM] = np.array_split(TrafficData['Dataset'][DATA_NUM], DATA_GROUPS)

# Transpose the grouped splits (interleave across datasets)
# Each element of `group_set` is a tuple of slices, one from each dataset
interleaved_groups = list(zip(
    TrafficData['Split']['30'],
    TrafficData['Split']['100'],
    TrafficData['Split']['70'],
    TrafficData['Split']['50']
))

# Concatenate groups vertically
TrafficData['Combined'] = pd.concat(
    [pd.concat(group_set, axis=0).reset_index(drop=True) for group_set in interleaved_groups],
    axis=0
).reset_index(drop=True)

print("Combined shape:", TrafficData['Combined'].shape)


Combined shape: (737280, 99)


In [ ]:
TrafficData['Train'] = {}
TrafficData['Train']['X'] = TrafficData['Combined'].iloc[:, 0:-1]
TrafficData['Train']['y'] = TrafficData['Combined'].iloc[:, -1]
print(TrafficData['Train']['X'].shape)
print(TrafficData['Train']['y'].shape)

TrafficData['Test'] = {}
TrafficData['Test']['X']=TrafficData['Dataset']['testing'].iloc[:, 0:-1]
TrafficData['Test']['y']=TrafficData['Dataset']['testing'].iloc[:, -1]
print(TrafficData['Test']['X'].shape)
print(TrafficData['Test']['y'].shape)

(737280, 98)
(737280,)
(120000, 98)
(120000,)


In [ ]:
scaler = MinMaxScaler()
model = scaler.fit(TrafficData['Train']['X'])
TrafficData['Train']['X'] = model.transform(TrafficData['Train']['X'])
TrafficData['Test']['X'] = model.transform(TrafficData['Test']['X'])

TrafficData['Train']['X'], TrafficData['Train']['y']= np.array(TrafficData['Train']['X']), np.array(TrafficData['Train']['y'])
print(type(TrafficData['Train']['X']),type(TrafficData['Train']['y']))
print(TrafficData['Train']['X'].shape,TrafficData['Train']['y'].shape)
TrafficData['Test']['X'], TrafficData['Test']['y']= np.array(TrafficData['Test']['X']), np.array(TrafficData['Test']['y'])
print(type(TrafficData['Test']['X']),type(TrafficData['Test']['y']))
print(TrafficData['Test']['X'].shape,TrafficData['Test']['y'].shape)

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(737280, 98) (737280,)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(120000, 98) (120000,)


In [ ]:
TrafficData['ROUNDS']={}
for ROUND in range(1, ROUNDS+1):
    TrafficData['ROUNDS'][ROUND]={}

SIZE_Demo = SIZE_ROUND
for ROUND in range(1,ROUNDS+1):
    if ROUND == 1:
        TrafficData['ROUNDS'][ROUND]['X']= TrafficData['Train']['X'][:SIZE_Demo]
        TrafficData['ROUNDS'][ROUND]['y']= TrafficData['Train']['y'][:SIZE_Demo]
    else:
        print((SIZE_Demo - SIZE_ROUND),SIZE_Demo)
        TrafficData['ROUNDS'][ROUND]['X']= TrafficData['Train']['X'][(SIZE_Demo - SIZE_ROUND):SIZE_Demo]
        TrafficData['ROUNDS'][ROUND]['y']= TrafficData['Train']['y'][(SIZE_Demo - SIZE_ROUND):SIZE_Demo]
    SIZE_Demo = SIZE_Demo + SIZE_ROUND
for ROUND in TrafficData['ROUNDS']:
    print("ROUND: ", ROUND, TrafficData['ROUNDS'][ROUND]['X'].shape, TrafficData['ROUNDS'][ROUND]['y'].shape)
print(SIZE_Demo, SIZE_ROUND)

3840 7680
7680 11520
11520 15360
15360 19200
19200 23040
23040 26880
26880 30720
ROUND:  1 (3840, 98) (3840,)
ROUND:  2 (3840, 98) (3840,)
ROUND:  3 (3840, 98) (3840,)
ROUND:  4 (3840, 98) (3840,)
ROUND:  5 (3840, 98) (3840,)
ROUND:  6 (3840, 98) (3840,)
ROUND:  7 (3840, 98) (3840,)
ROUND:  8 (3840, 98) (3840,)
34560 3840


In [ ]:
class ClassifierDataset(Dataset):
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
    def __len__ (self):
        return len(self.X_data)

In [ ]:
TrafficData['trainsets']={}
for ROUND in range(1, ROUNDS+1):
    TrafficData['trainsets'][ROUND]= ClassifierDataset(torch.from_numpy(TrafficData['ROUNDS'][ROUND]['X']).float(), torch.from_numpy(TrafficData['ROUNDS'][ROUND]['y']).long())
TrafficData['testset'] = ClassifierDataset(torch.from_numpy(TrafficData['Test']['X']).float(), torch.from_numpy(TrafficData['Test']['y']).long())

In [ ]:
def load_train(numberofclients, ROUND):    
    portion_size = int(BATCH_ROUND*BATCH_SIZE)
    num_portions = int(NUM_CLIENTS)
    portion_indices = []
    for i in range(num_portions):
        start_idx = i * portion_size
        end_idx = (i + 1) * portion_size
        portion_indices.append(list(range(start_idx, min(end_idx, SIZE_ROUND))))
    portion_datasets = [Subset(TrafficData['trainsets'][ROUND], indices) for indices in portion_indices]
    portion_loaders = [DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False) for dataset in portion_datasets]             
    return portion_loaders
def load_test(numberofclients):    
    testloader = DataLoader(TrafficData['testset'], batch_size=BATCH_SIZE, shuffle=False)
    return testloader

In [ ]:
Dataloaders = {}
for ROUND in range(1, ROUNDS+1):
    Dataloaders[ROUND] = load_train(NUM_CLIENTS, ROUND)
Dataloaders['Test'] = load_test(NUM_CLIENTS)

In [ ]:
for i, batch in enumerate(Dataloaders['Test']):
    batch_size = len(batch[0])  # Assuming the first element of the batch is the data
    print(f"Batch {i+1} size: {batch_size}")
    if batch_size != 64:
        print(f"Batch {i+1} does not contain 64 records.")
        break

Batch 1 size: 64
Batch 2 size: 64
Batch 3 size: 64
Batch 4 size: 64
Batch 5 size: 64
Batch 6 size: 64
Batch 7 size: 64
Batch 8 size: 64
Batch 9 size: 64
Batch 10 size: 64
Batch 11 size: 64
Batch 12 size: 64
Batch 13 size: 64
Batch 14 size: 64
Batch 15 size: 64
Batch 16 size: 64
Batch 17 size: 64
Batch 18 size: 64
Batch 19 size: 64
Batch 20 size: 64
Batch 21 size: 64
Batch 22 size: 64
Batch 23 size: 64
Batch 24 size: 64
Batch 25 size: 64
Batch 26 size: 64
Batch 27 size: 64
Batch 28 size: 64
Batch 29 size: 64
Batch 30 size: 64
Batch 31 size: 64
Batch 32 size: 64
Batch 33 size: 64
Batch 34 size: 64
Batch 35 size: 64
Batch 36 size: 64
Batch 37 size: 64
Batch 38 size: 64
Batch 39 size: 64
Batch 40 size: 64
Batch 41 size: 64
Batch 42 size: 64
Batch 43 size: 64
Batch 44 size: 64
Batch 45 size: 64
Batch 46 size: 64
Batch 47 size: 64
Batch 48 size: 64
Batch 49 size: 64
Batch 50 size: 64
Batch 51 size: 64
Batch 52 size: 64
Batch 53 size: 64
Batch 54 size: 64
Batch 55 size: 64
Batch 56 size: 64
B

Batch 85 size: 64
Batch 86 size: 64
Batch 87 size: 64
Batch 88 size: 64
Batch 89 size: 64
Batch 90 size: 64
Batch 91 size: 64
Batch 92 size: 64
Batch 93 size: 64
Batch 94 size: 64
Batch 95 size: 64
Batch 96 size: 64
Batch 97 size: 64
Batch 98 size: 64
Batch 99 size: 64
Batch 100 size: 64
Batch 101 size: 64
Batch 102 size: 64
Batch 103 size: 64
Batch 104 size: 64
Batch 105 size: 64
Batch 106 size: 64
Batch 107 size: 64
Batch 108 size: 64
Batch 109 size: 64
Batch 110 size: 64
Batch 111 size: 64
Batch 112 size: 64
Batch 113 size: 64
Batch 114 size: 64
Batch 115 size: 64
Batch 116 size: 64
Batch 117 size: 64
Batch 118 size: 64
Batch 119 size: 64
Batch 120 size: 64
Batch 121 size: 64
Batch 122 size: 64
Batch 123 size: 64
Batch 124 size: 64
Batch 125 size: 64
Batch 126 size: 64
Batch 127 size: 64
Batch 128 size: 64
Batch 129 size: 64
Batch 130 size: 64
Batch 131 size: 64
Batch 132 size: 64
Batch 133 size: 64
Batch 134 size: 64
Batch 135 size: 64
Batch 136 size: 64
Batch 137 size: 64
Batch 13

In [ ]:
for i, batch in enumerate(Dataloaders[5][0]):
    batch_size = len(batch[0])  # Assuming the first element of the batch is the data
    print(f"Batch {i+1} size: {batch_size}")
    if batch_size != 64:
        print(f"Batch {i+1} does not contain 64 records.")
        break

Batch 1 size: 64
Batch 2 size: 64
Batch 3 size: 64
Batch 4 size: 64
Batch 5 size: 64
Batch 6 size: 64


In [ ]:
from collections import Counter
for CLUSTER in range (1, 9):
    DEVICE_PERCENTAGE = []
    for DEVICE__ in range(0,3):
        for i, batch in enumerate(Dataloaders[CLUSTER][DEVICE__]):
            _, labels = batch
            class_counts = Counter(labels.numpy())
            total_records = sum(class_counts.values())
            class_0_count = class_counts.get(0, 0)
            percentage_class_0 = (class_0_count / total_records) * 100
            DEVICE_PERCENTAGE.append(percentage_class_0)
            # print(f"Batch {i+1}: {dict(class_counts)}")
            # print(f"Percentage of class 0: {percentage_class_0:.2f}%\n")
    # print(DEVICE_PERCENTAGE)        
    chunk_size = 6
    averages = [sum(DEVICE_PERCENTAGE[i:i + chunk_size]) / chunk_size for i in range(0, len(DEVICE_PERCENTAGE), chunk_size)]
    # print("Averages of every device:")
    # print(averages)
    chunk_size_4 = 4
    averages = [sum(averages[i:i + chunk_size_4]) / chunk_size_4 for i in range(0, len(averages), chunk_size_4)]
    # print("Averages of every 4 devices:")
    print(averages)

[21.6796875]
[48.6328125]
[75.0]
[32.2265625]
[51.7578125]
[41.796875]
[36.328125]
[68.03385416666667]


In [ ]:
del TrafficData

<font color='Red'>***Neural Network***</font>
---
--- 

In [ ]:
class Net(nn.Module):
    def __init__(self,
                 input_dim=98,
                 output_dim=15,
                 h1=64,
                 h2=32,
                 dropout=0.0,
                 activation="relu",
                 batch_norm=False):
        super(Net, self).__init__()

        # Define layers
        self.layer_1 = nn.Linear(input_dim, h1)
        self.layer_2 = nn.Linear(h1, h2)
        self.layer_out = nn.Linear(h2, output_dim)

        # Optional batch normalization
        self.bn1 = nn.BatchNorm1d(h1) if batch_norm else nn.Identity()
        self.bn2 = nn.BatchNorm1d(h2) if batch_norm else nn.Identity()

        # Activation mapping
        act_map = {
            "relu": nn.ReLU(),
            "gelu": nn.GELU(),
            "leakyrelu": nn.LeakyReLU(0.1)
        }
        self.act = act_map.get(activation, nn.ReLU())

        # Dropout
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.layer_1(x)
        x = self.bn1(x)
        x = self.act(x)
        x = self.dropout(x)

        x = self.layer_2(x)
        x = self.bn2(x)
        x = self.act(x)
        x = self.dropout(x)

        x = self.layer_out(x)
        return x


In [ ]:
# # ============================================================
# # Initialize and save a random starting model (Round 0)
# # ============================================================
# print("🔧 Initializing base model using current TRAIN_CFG...")

# Random = Net(
#     input_dim=98,
#     output_dim=15,
#     h1=TRAIN_CFG.get("hidden_size_1", 64),
#     h2=TRAIN_CFG.get("hidden_size_2", 32),
#     dropout=TRAIN_CFG.get("dropout", 0.0),
#     activation=TRAIN_CFG.get("activation", "relu"),
#     batch_norm=TRAIN_CFG.get("batch_norm", False),
# )

# # Show layer shapes for sanity check
# for param_tensor in Random.state_dict():
#     print(param_tensor, "\t", Random.state_dict()[param_tensor].size())

# # Ensure the output directory exists
# os.makedirs(PATH, exist_ok=True)

# # Save initial random model weights
# init_model_path = os.path.join(PATH, "0_Input_Random_model_Net.pth")
# torch.save(Random.state_dict(), init_model_path)

# print(f"✅ Saved initial model to {init_model_path}")


In [ ]:
def train(net, trainloader, epochs: int, lr: float, weight_decay: float = 0.0, verbose=True):
    """Train the network for one FL round using the current TRAIN_CFG parameters."""

    # --- Loss function (with optional label smoothing)
    criterion = nn.CrossEntropyLoss(label_smoothing=TRAIN_CFG.get("label_smoothing", 0.0))

    # --- Optimizer selection
    opt_name = TRAIN_CFG.get("optimizer", "adamw").lower()
    wd = weight_decay
    momentum = TRAIN_CFG.get("momentum", 0.0)

    if opt_name == "sgd":
        optimizer = torch.optim.SGD(net.parameters(), lr=lr, momentum=momentum, weight_decay=wd)
    elif opt_name == "rmsprop":
        optimizer = torch.optim.RMSprop(net.parameters(), lr=lr, momentum=momentum, weight_decay=wd)
    elif opt_name == "adam":
        optimizer = torch.optim.Adam(net.parameters(), lr=lr, weight_decay=wd)
    else:  # default: AdamW
        optimizer = torch.optim.AdamW(net.parameters(), lr=lr, weight_decay=wd)

    # --- Scheduler (optional)
    sched_type = TRAIN_CFG.get("scheduler", "none").lower()
    scheduler = None
    if sched_type == "step":
        scheduler = torch.optim.lr_scheduler.StepLR(
            optimizer,
            step_size=int(TRAIN_CFG.get("scheduler_step", 2)),
            gamma=float(TRAIN_CFG.get("scheduler_gamma", 0.9))
        )
    elif sched_type == "cosine":
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
            optimizer, T_max=epochs
        )
    elif sched_type == "plateau":
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, factor=float(TRAIN_CFG.get("scheduler_gamma", 0.9)), patience=2
        )

    # --- Training loop
    net.train()
    prediction_matrix, actual_matrix, acc_matrix, loss_matrix = [], [], [], []
    grad_clip = float(TRAIN_CFG.get("gradient_clip_norm", 0.0))
    early_stop_patience = int(TRAIN_CFG.get("early_stopping_patience", 0))
    best_loss = float("inf")
    patience_ctr = 0

    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for images, labels in trainloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(outputs, labels)
            loss.backward()

            # optional gradient clipping
            if grad_clip > 0:
                torch.nn.utils.clip_grad_norm_(net.parameters(), grad_clip)

            optimizer.step()
            epoch_loss += loss.item()
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
            prediction_matrix.append(torch.max(outputs.data, 1)[1].tolist())
            actual_matrix.append(labels.tolist())

        # --- epoch end
        epoch_loss /= max(1, len(trainloader))
        epoch_acc = correct / max(1, total)
        loss_matrix.append(epoch_loss)
        acc_matrix.append(epoch_acc)

        # Scheduler step
        if scheduler:
            if isinstance(scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau):
                scheduler.step(epoch_loss)
            else:
                scheduler.step()

        # Early stopping check
        if early_stop_patience > 0:
            if epoch_loss < best_loss:
                best_loss = epoch_loss
                patience_ctr = 0
            else:
                patience_ctr += 1
                if patience_ctr >= early_stop_patience:
                    if verbose:
                        print(f"⏹ Early stopping at epoch {epoch+1}")
                    break

    return prediction_matrix, actual_matrix, acc_matrix, loss_matrix


def test(net, testloader):
    """Evaluate model performance on validation/test data."""
    criterion = nn.CrossEntropyLoss(label_smoothing=TRAIN_CFG.get("label_smoothing", 0.0))
    correct, total, loss = 0, 0, 0.0
    net.eval()
    prediction_matrix, actual_matrix, acc_matrix, loss_matrix = [], [], [], []

    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            prediction_matrix.append(predicted.tolist())
            actual_matrix.append(labels.tolist())

    loss /= len(testloader.dataset)
    accuracy = correct / max(1, total)
    loss_matrix.append(loss)
    acc_matrix.append(accuracy)

    print(f"Evaluation: eval loss {loss:.6f}, eval accuracy {accuracy:.4f}")
    return loss, accuracy, prediction_matrix, actual_matrix, acc_matrix, loss_matrix


In [ ]:
prediction_dict= {}
actual_dict= {}
accuracy_dict= {}
loss_dict= {}

<font color='green'>***LLM***</font>
---
--- 

In [ ]:
# ============================================================
# LLM Reviewer/Optimizer (Gemma-first, synchronized JSON + DONE sentinel)
# ============================================================
import os, re, json, math, glob, time, torch
from functools import lru_cache
from statistics import mean
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# ---- Toggle offline/online
LLM_OFFLINE = False  # set True to force offline/cached mode

if LLM_OFFLINE:
    os.environ["TRANSFORMERS_OFFLINE"] = "1"
    os.environ["HF_HUB_OFFLINE"] = "1"
os.environ.setdefault("TRANSFORMERS_CACHE", "/tmp/hf_cache")
os.makedirs("/tmp/gemma_offload", exist_ok=True)

# ---- Model candidates (Gemma 3 preferred, then open fallbacks)
_GEMMA_CANDIDATES = [
    "google/gemma-3-4b-it",
    "microsoft/Phi-3-mini-4k-instruct",
]

def _load_first_available_model(candidates):
    last_err = None
    hf_token = os.environ.get("HF_TOKEN")
    for mid in candidates:
        try:
            print(f"Trying LLM: {mid}")
            tok = AutoTokenizer.from_pretrained(mid, use_fast=True, token=hf_token)
            if tok.pad_token is None:
                tok.pad_token = tok.eos_token
            device_llm = "cuda" if torch.cuda.is_available() else "cpu"
            dtype_llm = torch.float16 if device_llm == "cuda" else torch.float32
            llm = AutoModelForCausalLM.from_pretrained(
                mid,
                dtype=dtype_llm,
                device_map="auto",
                low_cpu_mem_usage=True,
                offload_folder="/tmp/gemma_offload",
                token=hf_token,
            )
            pipe = pipeline("text-generation", model=llm, tokenizer=tok, dtype=dtype_llm)
            print(f"✅ Loaded {mid}")
            return mid, pipe
        except Exception as e:
            print(f"❌ Failed {mid}: {e}")
            last_err = e
    raise RuntimeError(f"No candidate LLM could be loaded. Last error: {last_err}")

@lru_cache(maxsize=1)
def get_llm_pipe():
    return _load_first_available_model(tuple(_GEMMA_CANDIDATES))

# ---- Default configuration
DEFAULT_CONFIG = {
    "round": 0,
    "learning_rate": LEARNING_RATE,
    "epochs": EPOCHS,
    "batch_size": BATCH_SIZE,
    "optimizer": "adamw",
    "momentum": 0.0,
    "weight_decay": 0.0,
    "dropout": 0.0,
    "gradient_clip_norm": 0.0,
    "label_smoothing": 0.0,
    "scheduler": "none",
    "scheduler_step": 2,
    "scheduler_gamma": 0.9,
    "early_stopping_patience": 0,
    "fraction_fit": 1.0,
    "aggregation": "FedAvg",
    "hidden_size_1": 64,
    "hidden_size_2": 32,
    "activation": "relu",
    "batch_norm": False,
    "notes": "",
}

def save_config(cfg: dict, path: str):
    with open(path, "w") as f:
        json.dump(cfg, f, indent=2)

def load_or_create_config(path: str) -> dict:
    if os.path.exists(path):
        with open(path) as f:
            return json.load(f)
    save_config(DEFAULT_CONFIG, path)
    return dict(DEFAULT_CONFIG)

# ---- Merge and clamp LLM suggestions
def apply_llm_suggestions(current_cfg: dict, llm_json: dict) -> dict:
    import copy
    updated = copy.deepcopy(current_cfg)

    allowed_keys = {
        "learning_rate": float,
        "epochs": int,
        "weight_decay": float,
        "dropout": float,
        "batch_size": int,
        "optimizer": str,
        "momentum": float,
        "gradient_clip_norm": float,
        "label_smoothing": float,
        "scheduler": str,
        "scheduler_step": int,
        "scheduler_gamma": float,
        "early_stopping_patience": int,
        "fraction_fit": float,
        "aggregation": str,
        "hidden_size_1": int,
        "hidden_size_2": int,
        "activation": str,
        "batch_norm": lambda x: str(x).lower() == "true",
        "notes": str,
    }

    for k, caster in allowed_keys.items():
        if k in llm_json:
            try:
                updated[k] = caster(llm_json[k])
            except Exception:
                pass

    # Attach entire LLM output to notes when available, otherwise stringify llm_json
    raw_output = None
    for k in ("_raw_output", "raw_output", "full_output", "llm_output", "output"):
        if k in llm_json and llm_json[k]:
            raw_output = llm_json[k]
            break
    if raw_output is None:
        try:
            raw_output = json.dumps(llm_json, ensure_ascii=False)
        except Exception:
            raw_output = str(llm_json)
    updated["notes"] = str(raw_output)
    return updated

# ---- Text generation and JSON parsing helpers
_GEN_ARGS = dict(max_new_tokens=512, do_sample=False)

def _extract_json(text: str) -> dict:
    """Extract last valid JSON block even if truncated."""
    import json
    start = text.find("{")
    end = text.rfind("}")
    if start == -1 or end == -1 or end <= start:
        return {}
    snippet = text[start:end + 1]
    try:
        return json.loads(snippet)
    except json.JSONDecodeError:
        # Try trimming stepwise until it parses
        for cut in range(len(snippet) - 1, 0, -1):
            try:
                return json.loads(snippet[:cut] + "}")
            except Exception:
                continue
        return {}

def generate_until_done(pipe, prompt, max_wait=120, max_retries=5):
    """Generate text from LLM and wait until '### DONE' appears."""
    output = ""
    for attempt in range(max_retries):
        chunk = pipe(prompt, **_GEN_ARGS)[0]["generated_text"]
        output += chunk
        if "### DONE" in output:
            print("✅ LLM finished generating (### DONE detected).")
            output = output.split("### DONE")[0]
            return output.strip()
        print(f"⚠️ LLM output incomplete (attempt {attempt+1}/{max_retries}), retrying...")
        time.sleep(2)
    print("⚠️ Giving up waiting for DONE sentinel — using best-effort output.")
    return output.strip()

# ---- Historical config loading
def _detect_current_round_from_jsons(base_path: str) -> int:
    files = glob.glob(os.path.join(base_path, "train_config_round_*.json"))
    rounds = []
    for f in files:
        try:
            r = int(os.path.splitext(os.path.basename(f))[0].split("_")[-1])
            rounds.append(r)
        except Exception:
            pass
    return max(rounds) if rounds else 1

def _load_recent_configs(base_path: str, current_round: int, window: int = 5):
    cfgs = []
    for r in range(max(1, current_round - window), current_round):
        p = os.path.join(base_path, f"train_config_round_{r}.json")
        if os.path.exists(p):
            try:
                with open(p, "r") as f:
                    data = json.load(f)
                    data["round"] = r
                    cfgs.append(data)
            except Exception:
                pass
    return cfgs

# ---- Main LLM review + suggestion step
def llm_review_and_suggest_llm3(current_cfg: dict, metrics: dict, info: dict, *,
                                base_path: str = None, round_idx: int | None = None) -> dict:
    """
    Review training results and suggest improved hyperparameters.
    Waits until LLM finishes ('### DONE') before continuing.
    """
    model_id, pipe = get_llm_pipe()

    base_path = base_path or (globals().get("PATH") or ".")
    os.makedirs(base_path, exist_ok=True)
    if round_idx is None:
        round_idx = _detect_current_round_from_jsons(base_path)

    prev_cfgs = _load_recent_configs(base_path, round_idx, window=5)
    hist_summary = {
        "n_previous": len(prev_cfgs),
        "avg_lr": mean([c.get("learning_rate", 0.0) for c in prev_cfgs]) if prev_cfgs else None,
        "avg_epochs": mean([c.get("epochs", 0) for c in prev_cfgs]) if prev_cfgs else None,
        "avg_wd": mean([c.get("weight_decay", 0.0) for c in prev_cfgs]) if prev_cfgs else None,
        "avg_dropout": mean([c.get("dropout", 0.0) for c in prev_cfgs]) if prev_cfgs else None,
    }

    prompt = f"""
You are an expert federated learning optimization assistant integrated into an automated ML pipeline.
Your purpose is to improve generalization, validation precision, and accuracy across rounds of federated training.
You will be provided with:
  1️⃣ CURRENT_CONFIG — the current round’s hyperparameters,
  2️⃣ ROUND_METRICS — validation metrics for this round,
  3️⃣ DATA_INFO — dataset and hardware context,
  4️⃣ LAST_5_CONFIGS — up to five prior configurations for trend analysis.

# Instructions
- Analyze performance and trends.
- Suggest small, data-driven hyperparameter updates.
- Avoid overfitting; stability is more important than drastic improvements.
- Prefer conservative changes in learning rate and epochs.
- If val_loss ↑ while accuracy ↑ → add mild regularization (dropout or weight_decay).
- If val_loss ↓ but accuracy stagnates → slightly increase epochs or learning_rate.
- If both degrade → reduce learning_rate and add regularization.
- If metrics plateau for 3+ rounds → only minor exploration.

# STRICT OUTPUT FORMAT
Return **only** a single JSON object.
Never include markdown code fences (no ```json or ```).
Do not include explanations, text, comments, or reasoning outside the JSON object.

The JSON must follow this schema exactly:
{{
  "learning_rate": <float>,
  "epochs": <int>,
  "weight_decay": <float>,
  "dropout": <float>,
  "batch_size": <int>,
  "optimizer": <"adam" | "adamw" | "sgd">,
  "momentum": <float>,
  "gradient_clip_norm": <float>,
  "label_smoothing": <float>,
  "scheduler": <"none" | "StepLR" | "ReduceLROnPlateau" | "CosineAnnealing">,
  "scheduler_step": <int>,
  "scheduler_gamma": <float>,
  "early_stopping_patience": <int>,
  "fraction_fit": <float>,
  "aggregation": <"FedAvg" | "FedProx" | "mean">,
  "hidden_size_1": <int>,
  "hidden_size_2": <int>,
  "activation": <"relu" | "gelu" | "tanh" | "elu">,
  "batch_norm": <bool>,
  "notes": "<short rationale under 20 words>"
}}

CURRENT_CONFIG:
{json.dumps(current_cfg, indent=2)}

LAST_ROUND_METRICS:
{json.dumps(metrics, indent=2)}

DATA_INFO:
{json.dumps(info, indent=2)}

LAST_5_CONFIGS:
{json.dumps(prev_cfgs, indent=2)}

HISTORICAL_SUMMARY:
{json.dumps(hist_summary, indent=2)}

Return *only* the JSON object matching the above schema, nothing else.
# Your task:
Analyze all inputs and output ONLY the JSON object above — nothing else.
After the JSON, append a new line that says exactly without quotation marks: "### DONE"
"""

    # --- Generate output, wait until sentinel ---
    out = generate_until_done(pipe, prompt)

    # --- Show and save full output ---
    print("\n==============================")
    print(f"🧠 Full LLM Output (Round {round_idx})")
    print("==============================")
    print(out)
    print("==============================\n")

    with open(os.path.join(base_path, f"llm_raw_round_{round_idx}.txt"), "w") as f:
        f.write(out)

    # --- Parse JSON safely ---
    suggestions = _extract_json(out) or {}
    if not suggestions:
        print("⚠️ No valid JSON detected — keeping previous configuration.")
    else:
        print(f"✅ Parsed LLM suggestions ({len(suggestions)} keys):")
        print(json.dumps(suggestions, indent=2))

    return suggestions


In [ ]:
# ============================================================
# Live training config (LLM-overridable between rounds)
# ============================================================
TRAIN_CFG = {
    # Core
    "learning_rate": float(LEARNING_RATE),
    "epochs": int(EPOCHS),
    "batch_size": int(BATCH_SIZE),

    # Regularization
    "weight_decay": 0.0,
    "dropout": 0.0,
    "label_smoothing": 0.0,
    "gradient_clip_norm": 0.0,   # 0 = disabled

    # Optimizer / Scheduler
    "optimizer": "adamw",        # adam|adamw|sgd|rmsprop
    "momentum": 0.0,             # used only for SGD-like opts
    "scheduler": "none",         # none|cosine|step|plateau
    "scheduler_step": 2,         # for 'step'
    "scheduler_gamma": 0.9,      # for 'step'/'plateau'
    "early_stopping_patience": 0,

    # Federated knobs (used when wiring strategy)
    "fraction_fit": 1.0,         # 0.1..1.0

    # (Model structure keys are persisted but applied when instantiating Net)
    "hidden_size_1": 64,
    "hidden_size_2": 32,
    "activation": "relu",        # relu|gelu|leakyrelu
    "batch_norm": False,
}

def use_cfg_in_globals(cfg: dict):
    """
    Apply *safe mid-round* overrides to globals.
    Keep batch_size static mid-run (rebuild loaders if you want it live).
    """
    global LEARNING_RATE, EPOCHS
    LEARNING_RATE = float(cfg.get("learning_rate", LEARNING_RATE))
    EPOCHS       = int(cfg.get("epochs", EPOCHS))

    # Persisted in TRAIN_CFG for the rest of the pipeline to consume:
    # - weight_decay/dropout: used when building optimizer / Net()
    # - optimizer/scheduler fields: read inside your train() to build opt/sched
    # - gradient_clip_norm/label_smoothing: applied inside train() if > 0
    # - fraction_fit: wire into SaveModelStrategy(fraction_fit=TRAIN_CFG["fraction_fit"])
    # - hidden sizes/activation/batch_norm: apply when you instantiate Net(...)
    #   e.g., Net(h1=TRAIN_CFG["hidden_size_1"], h2=..., activation=..., batch_norm=..., dropout=...)


In [ ]:
# ============================================================
# Metrics + LLM review step (call this after each round)
# ============================================================
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def evaluate_full(model, testloader):
    model = model.to(DEVICE)
    model.eval()
    crit = nn.CrossEntropyLoss()
    all_preds, all_labels = [], []
    loss_sum, n_batches = 0.0, 0
    with torch.no_grad():
        for x, y in testloader:
            x, y = x.to(DEVICE), y.to(DEVICE)
            out = model(x)
            loss_sum += crit(out, y).item()
            n_batches += 1
            preds = torch.max(out, 1)[1]
            all_preds.extend(preds.cpu().numpy().tolist())
            all_labels.extend(y.cpu().numpy().tolist())

    loss = loss_sum / max(1, n_batches)
    acc = accuracy_score(all_labels, all_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(
        all_labels, all_preds, average="macro", zero_division=0
    )

    return {
        "val_loss": float(loss),
        "val_accuracy": float(acc),
        "precision_macro": float(precision),
        "recall_macro": float(recall),
        "f1_macro": float(f1),
    }


def llm_step_after_round(round_idx: int, model_for_eval: nn.Module):
    """
    Evaluate the round’s model, run LLM review, and save next-round JSON.
    Blocks execution until the LLM provides a valid JSON response or times out.
    """

    print(f"🧮 Evaluating global model (Round {round_idx}) ...")
    metrics = evaluate_full(model_for_eval, Dataloaders['Test'])

    print("📊 Validation Metrics:")
    for k, v in metrics.items():
        print(f"  {k}: {v:.4f}")

    # ------------------------------------------------------
    # Load or initialize round config
    # ------------------------------------------------------
    current_cfg_path = os.path.join(PATH, f"train_config_round_{round_idx}.json")
    current_cfg = load_or_create_config(current_cfg_path)

    current_cfg.update({
        "round": round_idx,
        "learning_rate": float(TRAIN_CFG.get("learning_rate", LEARNING_RATE)),
        "epochs": int(TRAIN_CFG.get("epochs", EPOCHS)),
        "weight_decay": float(TRAIN_CFG.get("weight_decay", 0.0)),
        "dropout": float(TRAIN_CFG.get("dropout", 0.0)),
        "batch_size": int(TRAIN_CFG.get("batch_size", BATCH_SIZE)),
        "notes": f"Auto-updated after round {round_idx}",
    })

    # ------------------------------------------------------
    # Build LLM context
    # ------------------------------------------------------
    data_info = {
        "num_clients": int(NUM_CLIENTS),
        "round_size": int(SIZE_ROUND),
        "batch_size": int(BATCH_SIZE),
        "n_features": 98,
        "n_classes": 15,
        "device": str(DEVICE),
    }

    # ------------------------------------------------------
    # Call LLM and wait for DONE sentinel
    # ------------------------------------------------------
    print("\n🤖 Requesting LLM optimization suggestions...")
    suggestions = {}
    for attempt in range(2):  # retry once if JSON fails
        suggestions = llm_review_and_suggest_llm3(
            current_cfg, metrics, data_info,
            base_path=PATH, round_idx=round_idx
        )
        if suggestions:
            break
        print(f"⚠️ Attempt {attempt+1} failed — LLM did not return valid JSON. Retrying...\n")

    if not suggestions:
        print("⚠️ LLM did not return valid JSON after retries — keeping current hyperparameters.")

    # ------------------------------------------------------
    # Merge + save next configuration
    # ------------------------------------------------------
    updated_cfg = apply_llm_suggestions(current_cfg, suggestions)
    next_round = round_idx + 1
    next_cfg_path = os.path.join(PATH, f"train_config_round_{next_round}.json")
    save_config(updated_cfg, next_cfg_path)

    print(f"\n✅ Wrote {next_cfg_path}")
    print("LLM suggestions (applied):")
    print(f"  learning_rate = {updated_cfg.get('learning_rate')}")
    print(f"  epochs        = {updated_cfg.get('epochs')}")
    print(f"  weight_decay  = {updated_cfg.get('weight_decay')}")
    print(f"  dropout       = {updated_cfg.get('dropout')}")
    print("------------------------------------------------------")

    # ------------------------------------------------------
    # Update live training parameters for next round
    # ------------------------------------------------------
    TRAIN_CFG.update({
        "learning_rate": float(updated_cfg.get("learning_rate", LEARNING_RATE)),
        "epochs": int(updated_cfg.get("epochs", EPOCHS)),
        "weight_decay": float(updated_cfg.get("weight_decay", 0.0)),
        "dropout": float(updated_cfg.get("dropout", 0.0)),
    })
    use_cfg_in_globals(TRAIN_CFG)
    print("🔄 Global TRAIN_CFG updated for next round.\n")


<font color='Brown'>***Federated Learning Classes***</font>
---
--- 

In [ ]:
def get_parameters(net) -> List[np.ndarray]:
    """Extract model parameters as a list of NumPy arrays (CPU-safe)."""
    return [val.detach().cpu().numpy().astype(np.float32) for _, val in net.state_dict().items()]

def set_parameters(net, parameters: List[np.ndarray]):
    """Load parameters from NumPy arrays into the given network safely."""
    # Ensure matching key order
    params_dict = zip(net.state_dict().keys(), parameters)
    # Convert arrays to tensors with the same dtype/device
    state_dict = OrderedDict({k: torch.tensor(v, dtype=torch.float32, device=next(net.parameters()).device)
                              for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)


In [ ]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, FL_Update):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.FL_Update = FL_Update

    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        # --- Read config (LLM-driven) ---
        local_epochs = int(config.get("local_epochs", 1))
        lr = float(config.get("learning_rate", LEARNING_RATE))
        wd = float(config.get("weight_decay", 0.0))
        clip_norm = float(config.get("gradient_clip_norm", 0.0))

        print(f"[Client {self.cid}, round {self.FL_Update}] fit, "
              f"epochs={local_epochs}, lr={lr}, wd={wd}, clip_norm={clip_norm}")

        # --- Load global weights ---
        set_parameters(self.net, parameters)

        # --- Train locally ---
        _1, _2, _3, _4 = train(
            self.net,
            self.trainloader,
            epochs=local_epochs,
            lr=lr,
            weight_decay=wd,
        )

        # --- Optional gradient clipping if requested ---
        if clip_norm > 0:
            torch.nn.utils.clip_grad_norm_(self.net.parameters(), clip_norm)

        # --- Log local performance for analysis ---
        prediction_dict[f'C{self.cid}R{self.FL_Update}'] = _1
        actual_dict[f'C{self.cid}R{self.FL_Update}'] = _2
        accuracy_dict[f'C{self.cid}R{self.FL_Update}'] = _3
        loss_dict[f'C{self.cid}R{self.FL_Update}'] = _4

        # --- Return updated model parameters ---
        return get_parameters(self.net), len(self.trainloader.dataset), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate skipped (centralized eval only)")
        return None


In [ ]:
def retrieve_and_sort_client_updates(global_model, round_number, client_ids, base_path=PATH):
    """Compare and rank client contributions for a given round based on update magnitude."""
    client_updates = {}
    for cid in client_ids:
        update_filename = os.path.join(base_path, f"C{cid}R{round_number}_update.pkl")
        if not os.path.exists(update_filename):
            print(f"⚠️ Missing update file for client {cid}, round {round_number}")
            continue
        with open(update_filename, 'rb') as f:
            client_updates[cid] = pickle.load(f)

    if not client_updates:
        print("⚠️ No client updates found.")
        return [], []

    client_contributions = {
        cid: calculate_weight_magnitude(global_model, update)
        for cid, update in client_updates.items()
    }

    sorted_clients = sorted(client_contributions.items(), key=lambda x: x[1], reverse=True)
    least_contributing_clients = sorted_clients[-3:] if len(sorted_clients) >= 3 else sorted_clients
    return sorted_clients, least_contributing_clients


def calculate_weight_magnitude(global_model, client_update):
    """Compute the L2 norm of parameter differences between global and client weights."""
    weight_diff = 0.0
    global_params = [p.detach().cpu().numpy() for p in global_model.parameters()]

    for gp, cu in zip(global_params, client_update):
        try:
            diff = np.linalg.norm(gp - cu)
            if not np.isnan(diff):
                weight_diff += diff
        except Exception:
            continue

    return float(weight_diff)


<font color='Brown'>***Clients Functions***</font>
---
--- 

In [ ]:
def General_Client():
    def client_fn(cid: int, Round: int) -> FlowerClient:
        clients_ids_list = TrainingListPerRound[Round]
        if int(cid) in clients_ids_list:
            # Instantiate client model using current LLM-tuned architecture
            net = Net(
                input_dim=98,
                output_dim=15,
                h1=int(TRAIN_CFG.get("hidden_size_1", 64)),
                h2=int(TRAIN_CFG.get("hidden_size_2", 32)),
                dropout=float(TRAIN_CFG.get("dropout", 0.0)),
                activation=str(TRAIN_CFG.get("activation", "relu")),
                batch_norm=bool(TRAIN_CFG.get("batch_norm", False)),
            ).to(DEVICE)

            trainloader = Dataloaders[Round][int(cid)]
            return FlowerClient(cid, net, trainloader, Round)
        else:
            raise ValueError(f"Client ID {cid} not found in the list for round {Round}")
    return client_fn


<font color='Brown'>***FL Strategy***</font>
---
--- 

In [ ]:
Global_Models = {}

class SaveModelStrategy(fl.server.strategy.FedAvg):
    def __init__(self, additional_argument, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.additional_argument = additional_argument

    def aggregate_fit(
        self,
        rnd: int,
        results: List[Tuple[fl.server.client_proxy.ClientProxy, fl.common.FitRes]],
        failures: List[BaseException],
    ) -> Optional[fl.common.NDArrays]:
        """Aggregate client updates, rebuild the global model with current TRAIN_CFG, and save checkpoint."""
        aggregated_parameters_tuple = super().aggregate_fit(rnd, results, failures)
        if aggregated_parameters_tuple is None:
            return None

        aggregated_parameters, _ = aggregated_parameters_tuple
        if aggregated_parameters is not None:
            # Convert FL parameters to ndarray list
            aggregated_weights: List[np.ndarray] = fl.common.parameters_to_ndarrays(aggregated_parameters)

            # --- Rebuild model using the LLM-controlled architecture ---
            model_cfg = {
                "input_dim": 98,
                "output_dim": 15,
                "h1": int(TRAIN_CFG.get("hidden_size_1", 64)),
                "h2": int(TRAIN_CFG.get("hidden_size_2", 32)),
                "dropout": float(TRAIN_CFG.get("dropout", 0.0)),
                "activation": str(TRAIN_CFG.get("activation", "relu")),
                "batch_norm": bool(TRAIN_CFG.get("batch_norm", False)),
            }
            Global_Models[self.additional_argument] = Net(**model_cfg)

            # Map FL weights into model’s state dict
            params_dict = zip(Global_Models[self.additional_argument].state_dict().keys(), aggregated_weights)
            state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
            Global_Models[self.additional_argument].load_state_dict(state_dict, strict=True)

            # Ensure path exists
            os.makedirs(PATH, exist_ok=True)

            # Save checkpoint for current round
            save_path = f"{PATH}/GlobalModel_{self.additional_argument}.pth"
            torch.save(Global_Models[self.additional_argument].state_dict(), save_path)
            print(f"💾 Saved aggregated global model: {save_path}")

        return aggregated_parameters_tuple


In [ ]:
def fit_config(server_round: int):
    """Return the current LLM-tuned configuration for this federated round."""
    return {
        # Round tracking
        "current_round": int(server_round),

        # Core training hyperparameters
        "local_epochs": int(TRAIN_CFG.get("epochs", EPOCHS)),
        "learning_rate": float(TRAIN_CFG.get("learning_rate", LEARNING_RATE)),
        "weight_decay": float(TRAIN_CFG.get("weight_decay", 0.0)),

        # Regularization
        "dropout": float(TRAIN_CFG.get("dropout", 0.0)),
        "label_smoothing": float(TRAIN_CFG.get("label_smoothing", 0.0)),
        "gradient_clip_norm": float(TRAIN_CFG.get("gradient_clip_norm", 0.0)),

        # Architecture awareness (for reference / logging)
        "hidden_size_1": int(TRAIN_CFG.get("hidden_size_1", 64)),
        "hidden_size_2": int(TRAIN_CFG.get("hidden_size_2", 32)),
        "activation": str(TRAIN_CFG.get("activation", "relu")),
        "batch_norm": bool(TRAIN_CFG.get("batch_norm", False)),

        # Optimizer & scheduler (kept for LLM-driven experimentation)
        "optimizer": str(TRAIN_CFG.get("optimizer", "adam")),
        "scheduler": str(TRAIN_CFG.get("scheduler", "none")),

        # Meta info
        "batch_size": int(TRAIN_CFG.get("batch_size", BATCH_SIZE)),
        "early_stopping_patience": int(TRAIN_CFG.get("early_stopping_patience", 0)),
        "notes": TRAIN_CFG.get("notes", f"Auto-updated from round {server_round}"),
    }


***Running the Generalized FL Round***
---
---

In [ ]:
# ============================================================
# Initialize and save a random starting model (Round 0)
# ============================================================
print("🔧 Initializing base model using current TRAIN_CFG...")

Random = Net(
    input_dim=98,
    output_dim=15,
    h1=TRAIN_CFG.get("hidden_size_1", 64),
    h2=TRAIN_CFG.get("hidden_size_2", 32),
    dropout=TRAIN_CFG.get("dropout", 0.0),
    activation=TRAIN_CFG.get("activation", "relu"),
    batch_norm=TRAIN_CFG.get("batch_norm", False),
)

# Show layer shapes for sanity check
for param_tensor in Random.state_dict():
    print(param_tensor, "\t", Random.state_dict()[param_tensor].size())

# Ensure the output directory exists
os.makedirs(PATH, exist_ok=True)

# Save initial random model weights
init_model_path = os.path.join(PATH, "0_Input_Random_model_Net.pth")
torch.save(Random.state_dict(), init_model_path)

print(f"✅ Saved initial model to {init_model_path}")


🔧 Initializing base model using current TRAIN_CFG...
layer_1.weight 	 torch.Size([64, 98])
layer_1.bias 	 torch.Size([64])
layer_2.weight 	 torch.Size([32, 64])
layer_2.bias 	 torch.Size([32])
layer_out.weight 	 torch.Size([15, 32])
layer_out.bias 	 torch.Size([15])
✅ Saved initial model to llm-test-2/0_Input_Random_model_Net.pth


In [ ]:
print("Loading Initial Global Model")

# torch.set_float32_matmul_precision("medium")  # or "high" for A100s (ENABLE FOR LONG RUNS)

# ------------------------------------------------------
# Initialize base model using the latest TRAIN_CFG
# (ensures architecture matches LLM-tuned parameters)
# ------------------------------------------------------
Global_Models[0] = Net(
    h1=int(TRAIN_CFG.get("hidden_size_1", 64)),
    h2=int(TRAIN_CFG.get("hidden_size_2", 32)),
    dropout=float(TRAIN_CFG.get("dropout", 0.0)),
    activation=str(TRAIN_CFG.get("activation", "relu")),
    batch_norm=bool(TRAIN_CFG.get("batch_norm", False)),
)

# ------------------------------------------------------
# Load existing checkpoint if available
# ------------------------------------------------------
model_path = f"{PATH}/0_Input_Random_model_Net.pth"
if os.path.exists(model_path):
    print("✅ Pretrained model found — loading weights.")
    Global_Models[0].load_state_dict(torch.load(model_path))
else:
    print("⚠️ No pretrained model found — starting fresh.")
Global_Models[0].train()

# ------------------------------------------------------
# Prepare client lists for all rounds
# ------------------------------------------------------
TrainingListPerRound = {rnd: [int(c) for c in range(NUM_CLIENTS)] for rnd in range(1, ROUNDS + 1)}

# ======================================================
# Federated Learning Rounds with LLM Review
# ======================================================
for Round in range(1, ROUNDS + 1):
    print("\n" + "=" * 70)
    print(f"🚀 Starting Federated Round {Round}")
    print("=" * 70)

    # ------------------------------------------------------
    # 1️⃣ Load (or create) LLM-tuned training configuration
    # ------------------------------------------------------
    cfg_path = os.path.join(PATH, f"train_config_round_{Round}.json")
    TRAIN_CFG = load_or_create_config(cfg_path)
    use_cfg_in_globals(TRAIN_CFG)

    print(f"[Round {Round}] Using TRAIN_CFG:")
    for k, v in TRAIN_CFG.items():
        print(f"  {k}: {v}")

    # ------------------------------------------------------
    # 2️⃣ Initialize Federated Strategy
    # ------------------------------------------------------
    strategy = SaveModelStrategy(
        fraction_fit=1.0,
        fraction_evaluate=0.0,
        min_fit_clients=2,
        min_evaluate_clients=0,
        min_available_clients=2,
        on_fit_config_fn=fit_config,
        initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Global_Models[Round - 1])),
        additional_argument=Round,
    )

    client_fn = General_Client()

    # ------------------------------------------------------
    # 3️⃣ Run Federated Simulation for 1 Round
    # ------------------------------------------------------
    fl.simulation.start_simulation(
        client_fn=lambda cid: client_fn(cid, int(Round)),
        num_clients=int(NUM_CLIENTS),
        config=fl.server.ServerConfig(num_rounds=1),
        client_resources={"num_cpus": 16, "num_gpus": 1},
        ray_init_args={"num_cpus": 16, "num_gpus": 1},
        strategy=strategy,
    )

    # ------------------------------------------------------
    # 4️⃣ Load Global Model for Post-Round Evaluation
    # ------------------------------------------------------
    print(f"\n🔁 Loading Global Model for Round {Round} ...")
    os.makedirs(PATH, exist_ok=True)
    model_path = f"{PATH}/GlobalModel_{Round}.pth"

    Global_Models[Round] = Net(
        h1=int(TRAIN_CFG.get("hidden_size_1", 64)),
        h2=int(TRAIN_CFG.get("hidden_size_2", 32)),
        dropout=float(TRAIN_CFG.get("dropout", 0.0)),
        activation=str(TRAIN_CFG.get("activation", "relu")),
        batch_norm=bool(TRAIN_CFG.get("batch_norm", False)),
    )

    if os.path.exists(model_path):
        Global_Models[Round].load_state_dict(torch.load(model_path))
        print(f"✅ Model loaded from {model_path}")
    else:
        print(f"⚠️ No model found at {model_path}, continuing with initialized weights.")

    # ------------------------------------------------------
    # 5️⃣ Evaluate and Invoke LLM Optimization
    # ------------------------------------------------------
    Global_Models[Round].eval()
    print(f"\n📊 Evaluating and optimizing Round {Round} with LLM ...")
    llm_step_after_round(Round, Global_Models[Round])

print("\n===== 🎯 Training + LLM Optimization Complete =====")
print("All models and JSON configs saved under:", PATH)


Loading Initial Global Model
✅ Pretrained model found — loading weights.

🚀 Starting Federated Round 1
[Round 1] Using TRAIN_CFG:
  round: 0
  learning_rate: 0.0025
  epochs: 10
  batch_size: 64
  optimizer: adamw
  momentum: 0.0
  weight_decay: 0.0
  dropout: 0.0
  gradient_clip_norm: 0.0
  label_smoothing: 0.0
  scheduler: none
  scheduler_step: 2
  scheduler_gamma: 0.9
  early_stopping_patience: 0
  fraction_fit: 1.0
  aggregation: FedAvg
  hidden_size_1: 64
  hidden_size_2: 32
  activation: relu
  batch_norm: False
  notes: 


2025-11-05 18:55:27,463	INFO worker.py:2012 -- Started a local Ray instance.
[2025-11-05 18:55:27,474 I 51726 51726] logging.cc:303: Set ray log level from environment variable RAY_BACKEND_LOG_LEVEL to 2


(DefaultActor pid=55903) [Client 6, round 1] fit, epochs=10, lr=0.0025, wd=0.0, clip_norm=0.0


(DefaultActor pid=55903) /home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/transformers/utils/hub.py:110: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
(DefaultActor pid=55903)   warnings.warn(


(DefaultActor pid=55903) [Client 2, round 1] fit, epochs=10, lr=0.0025, wd=0.0, clip_norm=0.0
(DefaultActor pid=55903) [Client 7, round 1] fit, epochs=10, lr=0.0025, wd=0.0, clip_norm=0.0
(DefaultActor pid=55903) [Client 5, round 1] fit, epochs=10, lr=0.0025, wd=0.0, clip_norm=0.0
(DefaultActor pid=55903) [Client 3, round 1] fit, epochs=10, lr=0.0025, wd=0.0, clip_norm=0.0
(DefaultActor pid=55903) [Client 8, round 1] fit, epochs=10, lr=0.0025, wd=0.0, clip_norm=0.0
(DefaultActor pid=55903) [Client 4, round 1] fit, epochs=10, lr=0.0025, wd=0.0, clip_norm=0.0
(DefaultActor pid=55903) [Client 9, round 1] fit, epochs=10, lr=0.0025, wd=0.0, clip_norm=0.0
(DefaultActor pid=55903) [Client 1, round 1] fit, epochs=10, lr=0.0025, wd=0.0, clip_norm=0.0
(DefaultActor pid=55903) [Client 0, round 1] fit, epochs=10, lr=0.0025, wd=0.0, clip_norm=0.0
💾 Saved aggregated global model: llm-test-2/GlobalModel_1.pth

🔁 Loading Global Model for Round 1 ...
✅ Model loaded from llm-test-2/GlobalModel_1.pth

📊 

...

(DefaultActor pid=61288) /home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/transformers/utils/hub.py:110: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
(DefaultActor pid=61288)   warnings.warn(


(DefaultActor pid=61288) [Client 1, round 2] fit, epochs=10, lr=0.0025, wd=0.0, clip_norm=0.0
(DefaultActor pid=61288) [Client 4, round 2] fit, epochs=10, lr=0.0025, wd=0.0, clip_norm=0.0
(DefaultActor pid=61288) [Client 5, round 2] fit, epochs=10, lr=0.0025, wd=0.0, clip_norm=0.0
(DefaultActor pid=61288) [Client 0, round 2] fit, epochs=10, lr=0.0025, wd=0.0, clip_norm=0.0
(DefaultActor pid=61288) [Client 7, round 2] fit, epochs=10, lr=0.0025, wd=0.0, clip_norm=0.0
(DefaultActor pid=61288) [Client 6, round 2] fit, epochs=10, lr=0.0025, wd=0.0, clip_norm=0.0
(DefaultActor pid=61288) [Client 8, round 2] fit, epochs=10, lr=0.0025, wd=0.0, clip_norm=0.0
(DefaultActor pid=61288) [Client 2, round 2] fit, epochs=10, lr=0.0025, wd=0.0, clip_norm=0.0
(DefaultActor pid=61288) [Client 3, round 2] fit, epochs=10, lr=0.0025, wd=0.0, clip_norm=0.0
💾 Saved aggregated global model: llm-test-2/GlobalModel_2.pth

🔁 Loading Global Model for Round 2 ...
✅ Model loaded from llm-test-2/GlobalModel_2.pth

📊 

(pid=gcs_server) [2025-11-05 18:57:13,085 E 60812 60812] (gcs_server) gcs_server.cc:302: Failed to establish connection to the event+metrics exporter agent. Events and metrics will not be exported. Exporter agent status: RpcError: Running out of retries to initialize the metrics agent. rpc_code: 14
(raylet) [2025-11-05 18:57:16,273 E 61236 61236] (raylet) main.cc:975: Failed to establish connection to the metrics exporter agent. Metrics will not be exported. Exporter agent status: RpcError: Running out of retries to initialize the metrics agent. rpc_code: 14
(DefaultActor pid=61288) [2025-11-05 18:57:22,487 E 61288 61676] core_worker_process.cc:825: Failed to establish connection to the metrics exporter agent. Metrics will not be exported. Exporter agent status: RpcError: Running out of retries to initialize the metrics agent. rpc_code: 14
[2025-11-05 18:57:22,792 E 51726 61285] core_worker_process.cc:825: Failed to establish connection to the metrics exporter agent. Metrics will not b

✅ LLM finished generating (### DONE detected).

🧠 Full LLM Output (Round 2)
You are an expert federated learning optimization assistant integrated into an automated ML pipeline.
Your purpose is to improve generalization, validation precision, and accuracy across rounds of federated training.
You will be provided with:
  1️⃣ CURRENT_CONFIG — the current round’s hyperparameters,
  2️⃣ ROUND_METRICS — validation metrics for this round,
  3️⃣ DATA_INFO — dataset and hardware context,
  4️⃣ LAST_5_CONFIGS — up to five prior configurations for trend analysis.

# Instructions
- Analyze performance and trends.
- Suggest small, data-driven hyperparameter updates.
- Avoid overfitting; stability is more important than drastic improvements.
- Prefer conservative changes in learning rate and epochs.
- If val_loss ↑ while accuracy ↑ → add mild regularization (dropout or weight_decay).
- If val_loss ↓ but accuracy stagnates → slightly increase epochs or learning_rate.
- If both degrade → reduce learn

KeyboardInterrupt: 

<font color='Grey'>***Performance Testing***</font>
---
--- 

In [ ]:
import os
import glob

# Define the directory and file pattern
directory = PATH + '/'
pattern = "GlobalModel_*.pth"

# Find all matching files
files = glob.glob(os.path.join(directory, pattern))

# Extract numbers from file names
numbers = []
for file in files:
    base_name = os.path.basename(file)
    num_str = base_name.replace("GlobalModel_", "").replace(".pth", "")
    try:
        numbers.append(int(num_str))
    except ValueError:
        pass

# Determine the maximum number
max_num = max(numbers) if numbers else 0
print(max_num)

# Use the max_num in a loop
for num in range(1, max_num + 1):
    file_path = f"{PATH}/GlobalModel_{num}.pth"
    if os.path.exists(file_path):
        # Load the file or perform any operation you need
        print(f"Loading {file_path}")
    else:
        print(f"File {file_path} does not exist")

In [ ]:
def view_pickle(path, G=None, suffix=None):
    print(G)
    if G is not None:
        rounds = [G]
    else:
        # Auto-detect how many Global_X files exist
        rounds = sorted({
            int(f.split("_")[1])
            for f in os.listdir(path)
            if f.startswith("Global_") and f.split("_")[1].isdigit()
        })

    suffixes = [suffix] if suffix else ["pred", "actual", "accurracy", "loss"]

    for g in rounds:
        for s in suffixes:
            filename = os.path.join(path, f"Global_{g}_{s}")
            if not os.path.exists(filename):
                print(f"Missing file: {filename}")
                continue
            try:
                with open(filename, "rb") as f:
                    data = pickle.load(f)
                print(f"File: {filename}")
                if isinstance(data, list) and len(data) > 2:
                    pprint(data[:2])
                    print(f"... ({len(data)} total items)")
                else:
                    pprint(data)
            except Exception as e:
                print(f"❌ Error reading {filename}: {e}")



In [ ]:

pred_test = {}
actual_test = {}
accuracy_test = {}
loss_test = {}
G = 0
txt_path = f"{PATH}/Global_{G}.txt"
for num in range(1, max_num+1):
    model = Net()
    model.load_state_dict(torch.load(f"{PATH}/GlobalModel_{num}.pth"))
    model.eval()
    
    prediction_matrix = []
    actual_matrix= []
    acc_matrix = []
    loss_matrix=[]
    G = G + 1
    criterion = nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    with torch.no_grad():
        for images, labels in Dataloaders['Test']:
            outputs = model(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            prediction_matrix.append(predicted.tolist())
            actual_matrix.append(labels.tolist())
    loss /= len(Dataloaders['Test'].dataset)
    accuracy = correct / total
    loss_matrix.append(loss)
    acc_matrix.append(accuracy) 

    pred_test[f'Global_{G}'] = prediction_matrix
    actual_test[f'Global_{G}'] = actual_matrix
    accuracy_test[f'Global_{G}'] = acc_matrix
    loss_test[f'Global_{G}'] = loss_matrix 

    filename = f'{PATH}/Global_{G}_pred'
    outfile = open(filename,'wb')
    pickle.dump(pred_test[f'Global_{G}'],outfile)
    outfile.close()

    filename = f'{PATH}/Global_{G}_actual'
    outfile = open(filename,'wb')
    pickle.dump(actual_test[f'Global_{G}'],outfile)
    outfile.close()

    filename = f'{PATH}/Global_{G}_accurracy'
    outfile = open(filename,'wb')
    pickle.dump(accuracy_test[f'Global_{G}'],outfile)
    outfile.close()

    filename = f'{PATH}/Global_{G}_loss'
    outfile = open(filename,'wb')
    pickle.dump(loss_test[f'Global_{G}'],outfile)
    outfile.close()
    
view_pickle(PATH)
    